In [ ]:
! pip install split-folders

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.preprocessing.image import ImageDataGenerator
import os,glob
from tensorflow.keras.applications import VGG16,InceptionV3
from tensorflow.keras.layers import AveragePooling2D, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
import splitfolders

In [ ]:
! mkdir natural_images_split

In [ ]:
PATH = '../input/natural-images/natural_images'

In [ ]:
splitfolders.ratio(PATH, output='./natural_images_split', seed=1337, ratio=(.7, .2, .1), group_prefix=None)

In [ ]:
train_dir = './natural_images_split/train'
val_dir = './natural_images_split/val'
test_dir = './natural_images_split/test'

In [ ]:
def get_files(dir):
  if not os.path.exists(dir):
    return 0
  count = 0
  for current_path,dirs,files in os.walk(dir):
    for dr in dirs:
      count += len(glob.glob(os.path.join(current_path,dr +'/*')))
  return count

In [ ]:
data_training = get_files(train_dir)
data_val = get_files(val_dir)
data_test = get_files(test_dir)
num_classes = len(glob.glob(train_dir + "/*"))

print('jumlah data training :',data_training)
print('jumlah data val :',data_val)
print('jumlah data testing :',data_test)
print('jumlah keals dataset :',num_classes)

In [ ]:
train_datagen = ImageDataGenerator(
                rescale = 1/255,
                horizontal_flip=True,
                zoom_range = 0.3
                )

test_datagen = ImageDataGenerator(
                rescale = 1/255
                )

val_datagen = ImageDataGenerator(
                rescale = 1/255,
                horizontal_flip=True,
                zoom_range = 0.3
                )

In [ ]:
image_width = 224
image_height = 224

train_generator = train_datagen.flow_from_directory(
                    train_dir,
                    target_size = (image_width,image_height),
                    color_mode = 'rgb'
                    )

val_generator = val_datagen.flow_from_directory(
                  val_dir,
                  target_size = (image_width,image_height),
                  color_mode = 'rgb'   
                  )

test_generator = test_datagen.flow_from_directory(
                  test_dir,
                  target_size=(image_width,image_height),
                  color_mode = 'rgb',
                  shuffle=False
                  )

In [ ]:
train_generator.class_indices

In [ ]:
DSClass = ['airplane','car','cat','dog','flower','fruit','motorbike','person']

# Model VGG16

In [ ]:
bModel = VGG16(include_top=False,weights='imagenet',input_tensor=Input(shape=(224,224,3)))
bModel.summary()

In [ ]:
hModel = bModel.output
hModel = AveragePooling2D(pool_size=(4,4))(hModel)
hModel = Flatten()(hModel)
hModel = Dense(128,activation='relu')(hModel)
hModel = Dropout(0.5)(hModel)
hModel = Dense(8,activation='softmax')(hModel)

model = Model(bModel.input,hModel)

for layer in bModel.layers:
  layer.trainable=False

model.summary()

In [ ]:
opt = keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=opt,
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
batch_size = 32

train = model.fit(train_generator,
                            epochs=10,
                            steps_per_epoch = train_generator.samples // batch_size,
                            validation_data = val_generator,
                            validation_steps = val_generator.samples // batch_size,
                            verbose = 1)

In [ ]:
acc = train.history['accuracy']
val_acc = train.history['val_accuracy']
loss = train.history['loss']
val_loss = train.history['val_loss']
epochs = range(10)

# Plot akurasi data taraining dan validation
plt.plot(epochs, acc, 'b', label='Akurasi training')
plt.plot(epochs, val_acc, 'r', label='Akurasi validasi')
plt.title('Akurasi Training dan Validasi')
plt.legend()



plt.show()

In [ ]:
plt.plot(epochs, loss, 'b', label='Loss training',linestyle='--')
plt.plot(epochs, val_loss, 'r', label='Loss validasi',linestyle='--')
plt.title('Akurasi Training dan Validasi')
plt.legend()
plt.show()

In [ ]:
model.save('TugasADL_VGG16.h5')

In [ ]:
model = keras.models.load_model('./TugasADL_VGG16.h5')

In [ ]:
pred= model.predict_generator(test_generator)
predicted_class_indices=np.argmax(pred,axis=1)
labels=(test_generator.class_indices)
labels2=dict((v,k) for k,v in labels.items())
predictions=[labels2[k] for k in predicted_class_indices]
#print(predicted_class_indices)
#print(labels)
#print(predictions)

In [ ]:
len(predicted_class_indices)
len(labels)
len(predictions)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

confusion_matrix(predicted_class_indices,test_generator.labels)

In [ ]:
accuracy_score(predicted_class_indices,test_generator.labels)

In [ ]:
print(classification_report(predicted_class_indices,test_generator.labels))

In [ ]:
from keras.preprocessing import image
import matplotlib.pyplot as plt

import numpy as np

def prepare(img_path):
  img = image.load_img(img_path, target_size=(224,224,3), color_mode='rgb')
  x = image.img_to_array(img)
  x = x/255
  return np.expand_dims(x, axis=0)

In [ ]:
citra_test = '../input/cat-and-dog/test_set/test_set/dogs/dog.4007.jpg'

In [ ]:
imgs = image.load_img(citra_test, target_size=(224,224,3), color_mode='rgb')
plt.imshow(imgs)

In [ ]:
pred= model.predict_generator(prepare(citra_test))
hasil_akhir=np.argmax(pred,axis=1)
hasil_akhir_2 = model.predict(prepare(citra_test))
index = np.argsort(hasil_akhir_2[0,:])

print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print('Diagnosis Penyakit pada tanaman Tomat')
print('         by ShiftAcademy             ')
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print('')
print('Hasil Deteksi :')
print('')
print('          ', DSClass[int(hasil_akhir)])

# Model InceptionV3

In [ ]:
bModel2 = InceptionV3(include_top=False,weights='imagenet',input_tensor=Input(shape=(224,224,3)))
bModel2.summary()

In [ ]:
hModel2 = bModel2.output
hModel2 = AveragePooling2D(pool_size=(4,4))(hModel2)
hModel2 = Flatten()(hModel2)
hModel2 = Dense(128,activation='relu')(hModel2)
hModel2 = Dropout(0.5)(hModel2)
hModel2 = Dense(8,activation='softmax')(hModel2)

model2 = Model(bModel2.input,hModel2)

for layer in bModel2.layers:
  layer.trainable=False

model2.summary()

In [ ]:
opt = keras.optimizers.Adam(lr=0.001)
model2.compile(optimizer=opt,
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

batch_size = 32

train2 = model2.fit(train_generator,
                            epochs=10,
                            steps_per_epoch = train_generator.samples // batch_size,
                            validation_data = val_generator,
                            validation_steps = val_generator.samples // batch_size,
                            verbose = 1)

In [ ]:
acc = train2.history['accuracy']
val_acc = train2.history['val_accuracy']
loss = train2.history['loss']
val_loss = train2.history['val_loss']
epochs = range(10)

# Plot akurasi data taraining dan validation
plt.plot(epochs, acc, 'b', label='Akurasi training')
plt.plot(epochs, val_acc, 'r', label='Akurasi validasi')
plt.title('Akurasi Training dan Validasi')
plt.legend()



plt.show()

In [ ]:
plt.plot(epochs, loss, 'b', label='Loss training',linestyle='--')
plt.plot(epochs, val_loss, 'r', label='Loss validasi',linestyle='--')
plt.title('Akurasi Training dan Validasi')
plt.legend()
plt.show()

In [ ]:
model2.save('TugasADL_InceptionV3.h5')

In [ ]:
model2 = keras.models.load_model('./TugasADL_InceptionV3.h5')

In [ ]:
pred= model2.predict_generator(test_generator)
predicted_class_indices=np.argmax(pred,axis=1)
labels=(test_generator.class_indices)
labels2=dict((v,k) for k,v in labels.items())
predictions=[labels2[k] for k in predicted_class_indices]
#print(predicted_class_indices)
#print(labels)
#print(predictions)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

confusion_matrix(predicted_class_indices,test_generator.labels)

In [ ]:
print(classification_report(predicted_class_indices,test_generator.labels))

In [ ]:
print(accuracy_score(predicted_class_indices,test_generator.labels))

# SelfModel

In [ ]:
model3 = Sequential()

model3.add(Conv2D(128, (3,3), input_shape=(224,224, 3), padding='same', strides=1, activation='relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))

model3.add(Conv2D(64, (3,3), padding='same', strides=1, activation='relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))

model3.add(Flatten())
model3.add(Dense(128, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(8, activation='softmax'))

model3.summary()

In [ ]:
opt = keras.optimizers.Adam(lr=0.001)
model3.compile(optimizer=opt,
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

batch_size = 32

train3 = model3.fit(train_generator,
                            epochs=10,
                            steps_per_epoch = train_generator.samples // batch_size,
                            validation_data = val_generator,
                            validation_steps = val_generator.samples // batch_size,
                            verbose = 1)

In [ ]:
predicted = model3.predict_classes(test_generator, verbose=1)

In [ ]:
predicts = predicted.flatten()

In [ ]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt


data = {'y_Actual':    test_generator.classes,
        'y_Predicted': predicts
        }

df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
confusion_matrix = pd.crosstab(df['y_Actual'], df['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])

sn.heatmap(confusion_matrix, annot=True, fmt='.0f')
plt.show()

In [ ]:
print(classification_report(predicts,test_generator.labels))

In [ ]:
print(accuracy_score(predicts,test_generator.labels))

In [ ]:
model3.save('TugasADL_SelfModel.h5')